# **Resume Parsing using AI(NLP)**




#### Resume parser is a powerful tool to parse the complete resume and see the necessary details you want to see. In this world where the employees are blooming, there are lots of people who apply in certain organisation but the hiring managers have certain limited amount of time, so resume parser can be the right step in order to simplify this process of getting the particular information from the resume.

We can develop this resume parsing using two approaches:  
(1) Traditional approach using regular expression and string matching  
(2) using natural language processing to extract information  


In this project, I'll be using both of them for extracting different sections.Also, we'll be using one of the powerful libraries in nlp, i.e;spacy.<br>  
SpaCy, a powerful open-source library for natural language processing (NLP) in Python, is a valuable tool in the context of resume parsing. It offers pre-trained models for tasks like named entity recognition (NER) and part-of-speech (POS) tagging, allowing it to effectively extract and categorize information from resumes.

In [71]:
!pip install spacy
!from spacy.lang.en import English
!python -m spacy download en_core_web_sm


/bin/bash: line 1: from: command not found
2023-08-04 15:39:29.426460: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Now to read our resume which is in pdf form, we will install **pdfminer** which just takes pdf as input and extracts all the text from it.  

---


We need to also install **re** which is shorthand for regular expression.
**Regular expression** can be used as a powerful tool to synthesize the text using some patterns(which could be written in regular expression).    

---



Also we will use **matcher** which is from spacy library, it helps in matching sequences of tokens, based on pattern rules.
The **Matcher** lets you find words and phrases using rules describing their token attributes. Rules can refer to token annotations (like the text or part-of-speech tags), as well as lexical attributes like Token.is_punct.



In [72]:
!pip install pdfminer.six


In [73]:
from pdfminer.high_level import extract_text
import re
import spacy
from spacy.matcher import Matcher

We will extract some basic information from the resume:

1.   Skills
2.   Name
3.   Contact Number
4.   E-mail





In [74]:
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

In [109]:
resume_paths = [r"/content/Resume2.pdf"]

for resume_path in resume_paths:
    text = extract_text_from_pdf(resume_path)

print(text)

Harsh Pathak 

Curious Entity 

A curious entity to learn and transform 

EDUCATION 
Undergraduate 
NIT Arunachal Pradesh 
12/2020 - P,  

Fundamental topics of computer science 

Computer Science and
engineering 

Higher Secondary(12th) 
Crystal International Public School 
05/2018 - 04/2020,  

85% 

Fundamental knowledge in
domain of Science(PCM) 

singhsima2000@gmail.com 

9898116490 

linkedin.com/in/harsh-pathak-198259230 

github.com/haraldpat 

SKILLS 

C/C++ 

Python 

JavaScript 

India 

Mathematics 

Problem Solving 

Data structure and Algorithm 

Java 

NodeJs 

ReactJs 

MongoDB 

MySQL 

Computer Architecture 

AI & ML 

India 

Computer Networking 

System Design 

DBMS 

Microsoft Oﬃce 

WORK EXPERIENCE 
INTERN 
Cotton University 
12/2022 - 05/2023,  
INTERNSHIP ON COMPUTATIONAL BIOLOGY AND BIOINFORMATICS 

Achievements/Tasks 

Worked on real life data(DNA and RNA sequences) from NCBI and
performed computations(machine learning algorithms) to understand the
data and p

We will make a list of skills of which we want to be compulsory to be hired in organistaion and pass in this function, this function uses regular expression  to find the skills whether they are present or not, if they are present it will append to skills list.


In [75]:
def extract_skills_from_resume(text, skills_list):
    skills = []

    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skills.append(skill)

    return skills

In [76]:
def extract_contact_number_from_resume(text):
    contact_number = None

    pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()

    return contact_number

In [77]:
def extract_email_from_resume(text):
    email = None

    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()

    return email

Now to find name we will use spacy, we will use matcher for that and a pattern is defined as we know name consists of one/two/three proper noums in sequence, so we will use to find pattern where part of speech(POS) is a proper noun.

In [103]:
def extract_name(resume_text):
    nlp = spacy.load("en_core_web_sm")
    matcher = Matcher(nlp.vocab)

    patterns = [
        [{'POS':'NOUN'},{'POS':'PROPN'}],
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
    ]

    for pattern in patterns:
        matcher.add('NAME', patterns=[pattern])

    doc = nlp(resume_text) #this converts the text to tokens
    matches = matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

    return None

In [100]:
if __name__ == '__main__':
    resume_paths = [r"/content/Resume2.pdf"]

    for resume_path in resume_paths:
        text = extract_text_from_pdf(resume_path)
        print("Resume:", resume_path)

        name = extract_name(text)
        if name:
            print("Name:", name)
        else:
            print("Name not found")

        contact_number = extract_contact_number_from_resume(text)
        if contact_number:
            print("Contact Number:", contact_number)
        else:
            print("Contact Number not found")

        email = extract_email_from_resume(text)
        if email:
            print("Email:", email)
        else:
            print("Email not found")

        skills_list = ['C/C++', 'Python', 'Machine Learning', 'Statistics', 'Math', 'JavaScript', 'mySql']
        extracted_skills = extract_skills_from_resume(text, skills_list)
        if extracted_skills:
            print("Skills:", extracted_skills)
        else:
            print("No skills found")

        print()

Resume: /content/Resume2.pdf
Name: Harsh Pathak
Contact Number: 9898116490
Email: singhsima2000@gmail.com
Skills: ['Python', 'Machine Learning', 'Statistics', 'JavaScript', 'mySql']



Now, we have gathered the skills of an individual we want which is being matchef from what we want.  
If the candidate has 50% of the skills required then he will be considered for further evaluation.

In [91]:
if len(extracted_skills)>=(len(skills_list)/2):
  print(f'{name}:ACCEPTED')
else: print(f'{name}:NOT ACCEPTED')

Harsh Pathak:ACCEPTED


Now we will feed many resumes to this program and check which of them are perfect candidate.


In [108]:
resume_list = [[r"/content/Resume1.pdf"],[r"/content/Resume2.pdf"],
               [r"/content/Resume3.pdf"],[r"/content/Resume4.pdf"]]
for resume in resume_list:
  for resume_path in resume:
        text = extract_text_from_pdf(resume_path)
        name = extract_name(text)
        extracted_skills = extract_skills_from_resume(text, skills_list)

        #Name
        print(name)

        #Conatact Number
        contact_number = extract_contact_number_from_resume(text)
        if contact_number:
            print("Contact Number:", contact_number)
        else:
            print("Contact Number not found")

        #E-Mail
        email = extract_email_from_resume(text)
        if email:
            print("Email:", email)
        else:
            print("Email not found")

        #MATCHED SKILLS
        if extracted_skills:
            print("Skills:", extracted_skills)
        else:
            print("No skills found")

        #FINAL STATUS
        if len(extracted_skills)>=(len(skills_list)/2):
            print(f'STATUS : ACCEPTED')
        else: print(f'STATUS : NOT ACCEPTED')
        print()

BABURAM YADAV
Contact Number: 917376356928
Email: baburamyadav2690@gmail.com
Skills: ['Python', 'Machine Learning', 'Statistics', 'JavaScript']
STATUS : ACCEPTED

Harsh Pathak
Contact Number: 9898116490
Email: singhsima2000@gmail.com
Skills: ['Python', 'Machine Learning', 'Statistics', 'JavaScript', 'mySql']
STATUS : ACCEPTED

Patel Parag
Contact Number: 8320134512
Email: paragpagtel2908483@gmail.com
No skills found
STATUS : NOT ACCEPTED

Sonu Paswan
Contact Number not found
Email: paswansonu578@gmail.com
Skills: ['JavaScript', 'mySql']
STATUS : NOT ACCEPTED

